In [1]:
import os
import json
import glob
import numpy as np
import pandas as pd
import open3d as o3d
import meshplot as mp

In [2]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir, "data")
out_dir = os.path.join(base_dir, "results")

In [3]:
train_files = glob.glob(os.path.join(data_dir, "original", "train", "*", "*.obj"))
valid_files = glob.glob(os.path.join(data_dir, "original", "val", "*", "*.obj"))
len(train_files), len(valid_files)

(7003, 1088)

# file I/O

In [4]:
def read_objfile(file_path):
    vertices = []
    normals = []
    faces = []
    
    with open(file_path) as fr:
        for line in fr:
            data = line.split()
            if len(data) > 0:
                if data[0] == "v":
                    vertices.append(data[1:])
                elif data[0] == "vn":
                    normals.append(data[1:])
                elif data[0] == "f":
                    face = np.array([
                        [int(p.split("/")[0]), int(p.split("/")[2])]
                        for p in data[1:]
                    ]) - 1
                    faces.append(face)
    
    vertices = np.array(vertices, dtype=np.float32)
    normals = np.array(normals, dtype=np.float32)
    return vertices, normals, faces

In [5]:
def read_objfile_for_validate(file_path, return_o3d=False):
    # only for develop-time validation purpose.
    # this func force to load .obj file as triangle-mesh.
    
    obj = o3d.io.read_triangle_mesh(file_path)
    if return_o3d:
        return obj
    else:
        v = np.asarray(obj.vertices, dtype=np.float32)
        f = np.asarray(obj.triangles, dtype=np.int32)
        return v, f

In [6]:
def write_objfile(file_path, vertices, normals, faces):
    # write .obj file input-obj-style (mainly, header string is copy and paste).
    
    with open(file_path, "w") as fw:
        print("# Blender v2.82 (sub 7) OBJ File: ''", file=fw)
        print("# www.blender.org", file=fw)
        print("o test", file=fw)
        
        for v in vertices:
            print("v " + " ".join([str(c) for c in v]), file=fw)
        print("# {} vertices\n".format(len(vertices)), file=fw)
        
        for n in normals:
            print("vn " + " ".join([str(c) for c in n]), file=fw)
        print("# {} normals\n".format(len(normals)), file=fw)
            
        for f in faces:
            print("f " + " ".join(["{}//{}".format(c[0]+1, c[1]+1) for c in f]), file=fw)
        print("# {} faces\n".format(len(faces)), file=fw)
        
        print("# End of File", file=fw)

In [7]:
def validate_pipeline(v, n, f, out_dir):
    temp_path = os.path.join(out_dir, "temp.obj")
    write_objfile(temp_path, v, n, f)
    v_valid, f_valid = read_objfile_for_validate(temp_path)
    print(v_valid.shape, f_valid.shape)
    mp.plot(v_valid, f_valid)

In [8]:
vertices, normals, faces = read_objfile(train_files[0])
vertices.shape, normals.shape, len(faces)

((224, 3), (135, 3), 160)

In [9]:
validate_pipeline(vertices, normals, faces, out_dir)

(768, 3) (448, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# coordinate quantization

In [10]:
def bit_quantization(vertices, bit=8, v_min=-1., v_max=1.):
    # vertices must have values between -1 to 1.
    dynamic_range = 2 ** bit - 1
    discrete_interval = (v_max-v_min) / (dynamic_range)#dynamic_range
    offset = (dynamic_range) / 2
    
    vertices = vertices / discrete_interval + offset
    vertices = np.clip(vertices, 0, dynamic_range-1)
    return vertices.astype(np.int32)

In [11]:
v_quantized = bit_quantization(vertices)
v_quantized

array([[166, 108, 166],
       [ 88, 121, 166],
       [ 88, 108, 166],
       [123, 121, 166],
       [ 88, 108,  88],
       [166, 121,  88],
       [166, 108,  88],
       [131, 121, 166],
       [123, 121, 164],
       [ 88, 121,  88],
       [166, 121, 166],
       [131, 121,  88],
       [123, 153, 166],
       [ 90, 121, 164],
       [ 90, 121,  90],
       [123, 121,  88],
       [164, 121,  90],
       [131, 121,  90],
       [164, 121, 164],
       [131, 153, 166],
       [123, 153, 164],
       [131, 153,  88],
       [131, 121, 164],
       [131, 153, 164],
       [123, 154, 166],
       [123, 121,  90],
       [123, 153,  88],
       [131, 153,  90],
       [131, 154, 166],
       [123, 154, 164],
       [123, 153,  90],
       [131, 154,  88],
       [131, 154, 164],
       [123, 155, 164],
       [123, 154,  88],
       [131, 154,  90],
       [123, 155, 165],
       [131, 155, 164],
       [123, 154,  90],
       [131, 155,  89],
       [131, 155, 165],
       [123, 156

In [12]:
validate_pipeline(v_quantized, normals, faces, out_dir)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(712, 3) (408, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

# reduce points in the same grid

In [13]:
def redirect_same_vertices(vertices, faces):
    faces_with_coord = []
    for face in faces:
        faces_with_coord.append([[tuple(vertices[v_idx]), f_idx] for v_idx, f_idx in face])
    
    coord_to_minimum_vertex = {}
    new_vertices = []
    cnt_new_vertices = 0
    for vertex in vertices:
        vertex_key = tuple(vertex)
        
        if vertex_key not in coord_to_minimum_vertex.keys():
            coord_to_minimum_vertex[vertex_key] = cnt_new_vertices
            new_vertices.append(vertex)
            cnt_new_vertices += 1
    
    new_faces = []
    for face in faces_with_coord:
        face = np.array([
            [coord_to_minimum_vertex[coord], f_idx] for coord, f_idx in face
        ])
        new_faces.append(face)
    
    return np.stack(new_vertices), new_faces

In [14]:
v_redirected, f_redirected = redirect_same_vertices(v_quantized, faces)
v_redirected.shape, len(f_redirected)

((204, 3), 160)

In [21]:
validate_pipeline(v_redirected, normals, f_redirected, out_dir)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(712, 3) (408, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

# vertex/face sorting

In [22]:
def reorder_vertices(vertices):
    indeces = np.lexsort(vertices.T[::-1])[::-1]
    return vertices[indeces], indeces

In [23]:
v_reordered, sort_v_ids = reorder_vertices(v_redirected)

In [24]:
def reorder_faces(faces, sort_v_ids, pad_id=-1):
    # apply sorted vertice-id and sort in-face-triple values.
    
    faces_ids = []
    faces_sorted = []
    for f in faces:
        f = np.stack([
            np.concatenate([np.where(sort_v_ids==v_idx)[0], np.array([n_idx])])
            for v_idx, n_idx in f
        ])
        f_ids = f[:, 0]
        
        max_idx = np.argmax(f_ids)
        sort_ids = np.arange(len(f_ids))
        sort_ids = np.concatenate([
            sort_ids[max_idx:], sort_ids[:max_idx]
        ])
        faces_ids.append(f_ids[sort_ids])
        faces_sorted.append(f[sort_ids])
        
    # padding for lexical sorting.
    max_length = max([len(f) for f in faces_ids])
    faces_ids = np.array([
        np.concatenate([f, np.array([pad_id]*(max_length-len(f)))]) 
        for f in faces_ids
    ])
    
    # lexical sort over face triples.
    indeces = np.lexsort(faces_ids.T[::-1])[::-1]
    faces_sorted = [faces_sorted[idx] for idx in indeces]
    return faces_sorted

In [25]:
f_reordered = reorder_faces(f_redirected, sort_v_ids)

In [26]:
validate_pipeline(v_reordered, normals, f_reordered, out_dir)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(712, 3) (406, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

# loading pipeline

In [27]:
def load_pipeline(file_path, bit=8, remove_normal_ids=True):
    vs, ns, fs = read_objfile(file_path)
    
    vs = bit_quantization(vs, bit=bit)
    vs, fs = redirect_same_vertices(vs, fs)
    
    vs, ids = reorder_vertices(vs)
    fs = reorder_faces(fs, ids)
    
    if remove_normal_ids:
        fs = [f[:, 0] for f in fs]
        
    return vs, ns, fs

In [28]:
vs, ns, fs = load_pipeline(train_files[4], remove_normal_ids=False)

In [29]:
validate_pipeline(vs, ns, fs, out_dir)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
(123, 3) (97, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

# preparation of dataset

In [30]:
classes = ["basket", "chair", "lamp", "sofa", "table"]

In [32]:
train_info = []
for class_ in classes:
    print(class_)
    class_datas = []
    
    for file_path in train_files:
        if file_path.split("/")[-2] == class_:
            vs, ns, fs = load_pipeline(file_path)
            class_datas.append({
                "vertices": vs.tolist(),
                "faces": [f.tolist() for f in fs],
            })
            train_info.append({
                "vertices": sum([len(v) for v in vs]),
                "faces_sum": sum([len(f) for f in fs]),
                "faces_num": len(fs),
                "faces_points": max([len(f) for f in fs]),
            })
            
    with open(os.path.join(data_dir, "preprocessed", "train", class_+".json"), "w") as fw:
        json.dump(class_datas, fw, indent=4)

basket
chair
lamp
sofa
table


In [33]:
test_info = []
for class_ in classes:
    print(class_)
    class_datas = []
    
    for file_path in valid_files:
        if file_path.split("/")[-2] == class_:
            vs, ns, fs = load_pipeline(file_path)
            class_datas.append({
                "vertices": vs.tolist(),
                "faces": [f.tolist() for f in fs],
            })
            test_info.append({
                "vertices": sum([len(v) for v in vs]),
                "faces_sum": sum([len(f) for f in fs]),
                "faces_num": len(fs),
                "faces_points": max([len(f) for f in fs]),
            })
            
    with open(os.path.join(data_dir, "preprocessed", "valid", class_+".json"), "w") as fw:
        json.dump(class_datas, fw, indent=4)

basket
chair
lamp
sofa
table


In [34]:
train_info_df = pd.DataFrame(train_info)
train_info_df

,vertices,faces_sum,faces_num,faces_points
0,612,768,160,56
1,186,232,45,11
2,192,2424,601,24
3,249,278,54,23
4,273,148,15,65
...,...,...,...,...
6998,1008,1100,201,62
6999,1221,2086,363,63
7000,204,391,96,8
7001,123,176,37,14


In [35]:
test_info_df = pd.DataFrame(test_info)
test_info_df

,vertices,faces_sum,faces_num,faces_points
0,297,712,184,13
1,378,298,45,84
2,360,416,77,48
3,912,1200,290,24
4,1140,1102,164,183
...,...,...,...,...
1083,1056,1404,270,42
1084,96,106,23,8
1085,222,282,67,8
1086,270,380,71,29


In [36]:
print(train_info_df.max())
print("="*20)
print(test_info_df.max())

vertices        2346
faces_sum       3862
faces_num       1246
faces_points     330
dtype: int64
vertices        2292
faces_sum       3504
faces_num       1123
faces_points     257
dtype: int64


In [38]:
train_info_df.to_csv(os.path.join(out_dir, "statistics", "train_info.csv"))
test_info_df.to_csv(os.path.join(out_dir, "statistics", "test_info.csv"))

# check dataset

In [39]:
with open(os.path.join(data_dir, "preprocessed", "train", classes[0]+".json")) as fr:
    train = json.load(fr)
    
with open(os.path.join(data_dir, "preprocessed", "valid", classes[0]+".json")) as fr:
    valid = json.load(fr)
    
print(len(train), len(valid))

50 6


In [40]:
{k: v[:10] for k, v in train[0].items()}

{'vertices': [[166, 121, 166],
  [166, 121, 88],
  [166, 108, 166],
  [166, 108, 88],
  [165, 106, 165],
  [165, 106, 89],
  [165, 104, 165],
  [165, 104, 89],
  [165, 103, 165],
  [165, 103, 89]],
 'faces': [[203, 202, 200, 201],
  [203, 201, 147, 143, 97, 101, 1, 3],
  [203, 195, 194, 202],
  [203, 3, 5, 195],
  [202, 194, 4, 2],
  [202, 2, 0, 98, 94, 140, 144, 200],
  [201, 200, 144, 145, 184, 185, 146, 147],
  [199, 198, 196, 197],
  [199, 197, 7, 9],
  [199, 193, 192, 198]]}

In [41]:
{k: v[:10] for k, v in valid[0].items()}

{'vertices': [[164, 161, 158],
  [164, 161, 96],
  [164, 160, 159],
  [164, 160, 95],
  [164, 98, 159],
  [164, 98, 95],
  [163, 163, 158],
  [163, 163, 96],
  [163, 162, 158],
  [163, 162, 96]],
 'faces': [[98, 96, 95, 97],
  [98, 76, 73, 97],
  [98, 76, 72, 96],
  [97, 95, 71, 73],
  [96, 96, 72, 72],
  [96, 95, 95, 96],
  [96, 94, 93, 95],
  [96, 72, 65, 94],
  [95, 93, 64, 71],
  [95, 71, 71, 95]]}